## This script transforms the BDD100K dataset into Yolov8 format and filter specific classes.

__[Berkeley Deep Drive Dataset (BDD100K)](https://www.vis.xyz/bdd100k/)__: A Diverse Driving Dataset for Heterogeneous Multitask Learning (Images 100K) is a dataset for instance segmentation, semantic segmentation, object detection, and identification tasks. The dataset can be downloaded from __[here](https://dl.cv.ethz.ch/bdd100k/data/)__

BDD 100K has 10 classes for object detection
1. pedestrian
2. rider
3. car
4. truck
5. bus
6. train
7. motorcycle
8. bicycle
9. traffic light
10. traffic sign

### Filter specific Class
First we filter specific labels (in this case, traffic signs) from the BDD dataset by reading the original labels from a JSON file, extracting only the labels related to traffic signs, copying the corresponding images to a new directory, and saving the filtered labels into a new JSON file. It ensures that only the relevant traffic sign data is retained and all other classes are removed.

In [ ]:
import json
import os
import shutil

# Paths to the directories and files
image_dir = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/images/100k/val/"
label_file = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/labels/det_val.json"
output_label_file = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/labels/det_val_traffic_signs.json"
output_image_dir = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/images/100k/val_traffic_signs"

# Create the output directory if it doesn't exist
os.makedirs(output_image_dir, exist_ok=True)

# Load the JSON file
with open(label_file, 'r') as file:
    try:
        data = json.load(file)
    except json.JSONDecodeError as e:
        print(f"Error loading JSON file: {e}")
        raise

# Filter the labels
filtered_data = []
for item in data:
    try:
        if 'labels' in item:  # Check if 'labels' key exists
            traffic_sign_labels = [label for label in item['labels'] if label.get('category') == 'traffic sign']
            if traffic_sign_labels:
                filtered_labels = [{
                    'category': label['category'],
                    'box2d': label['box2d']
                } for label in traffic_sign_labels]

                filtered_data.append({
                    'name': item['name'],
                    'labels': filtered_labels,
                    'attributes': item.get('attributes', {})  # Include attributes if present
                })

                # Copy the corresponding image
                src_image_path = os.path.join(image_dir, item['name'])
                dst_image_path = os.path.join(output_image_dir, item['name'])
                if os.path.exists(src_image_path):
                    shutil.copy(src_image_path, dst_image_path)
        else:
            print(f"No 'labels' key found in item: {item['name']}")
    except KeyError as e:
        print(f"Error processing item {item['name']}: {e}")

# Save the filtered labels into a new JSON file
with open(output_label_file, 'w') as file:
    json.dump(filtered_data, file, indent=4)

print(f"Filtered labels saved to {output_label_file}")
print(f"Images copied to {output_image_dir}")


### Remove Extra images
Then we will Delete all the unnecessary images that do not contain our labels(traffic signs). For that a list of image names will be generated by the the .json of filtered labels and then all the images will be deleted that will not be present in that list. 

In [3]:
import json
import os

# Path to the directory containing images
image_dir = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/images/100k/train/"

# Path to val.json file
val_json_file = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/labels_csv/train.json"

# Load val.json to get list of image names
with open(val_json_file, 'r') as file:
    try:
        val_data = json.load(file)
        val_image_names = [item['name'] for item in val_data]
    except json.JSONDecodeError as e:
        print(f"Error loading JSON file: {e}")
        exit(1)

# Iterate through images in the directory
deleted_count = 0
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        if filename not in val_image_names:
            file_path = os.path.join(image_dir, filename)
            try:
                os.remove(file_path)
                deleted_count += 1
                print(f"Deleted: {filename}")
            except OSError as e:
                print(f"Error deleting {filename}: {e}")

print(f"Total images deleted: {deleted_count}")


Total images deleted: 0


### Convert BDD dataset to Yolo format

A typical Yolov8 format looks like this
```
dataset/
│
├── images/
│   ├── train/
│   ├── val/
│   └── test/    # Optional
│
├── labels/
│   ├── train/
│   ├── val/
│   └── test/    # Optional
│
└── data.yaml
```
The labels directory mirrors the images directory structure and contains the label files corresponding to each image. Each label file has the same name as its corresponding image file but with a .txt extension. These label files contain the annotation data in YOLO format.

Each label file contains annotations for a single image, with each line representing one object. The format for each line is

` class_id x_center y_center width height `

In [30]:
import json
import os

def convert_to_yolo_format(json_file, image_dir, output_dir):
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    for item in data:
        image_name = item['name']
        image_path = os.path.join(image_dir, image_name)
        
        if not os.path.exists(image_path):
            print(f"Image file {image_path} not found. Skipping.")
            continue
        
        image_width, image_height = get_image_size(image_path)
        
        if image_width == 0 or image_height == 0:
            print(f"Image {image_path} has invalid dimensions. Skipping.")
            continue
        
        txt_filename = os.path.splitext(image_name)[0] + ".txt"
        txt_path = os.path.join(output_dir, txt_filename)
        
        with open(txt_path, 'w') as txt_file:
            for label in item['labels']:
                category = label['category']
                box = label['box2d']
                x1, y1 = box['x1'], box['y1']
                x2, y2 = box['x2'], box['y2']
                
                # Calculate center and dimensions relative to image size
                center_x = (x1 + x2) / 2 / image_width
                center_y = (y1 + y2) / 2 / image_height
                width = (x2 - x1) / image_width
                height = (y2 - y1) / image_height
                
                # Write to YOLO format text file
                txt_file.write(f"{category} {center_x} {center_y} {width} {height}\n")

def get_image_size(image_path):
    try:
        from PIL import Image
        with Image.open(image_path) as img:
            width, height = img.size
            return width, height
    except Exception as e:
        print(f"Error getting image size for {image_path}: {e}")
        return 0, 0

# Paths to directories and files
image_dir_train = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/images/100k/train/"
image_dir_val = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/images/100k/val/"
train_json_file = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/labels/train.json"
val_json_file = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/labels/val.json"
output_dir_train = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd_yolo/train/"
output_dir_val = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd_yolo/val/"

# Convert train set
convert_to_yolo_format(train_json_file, image_dir_train, output_dir_train)

# Convert validation set
convert_to_yolo_format(val_json_file, image_dir_val, output_dir_val)

print("Conversion to YOLO format completed.")

Conversion to YOLO format completed.


### Label Replacement
Once the all the .txt files are generated with required yolo format, their class_id remains as string. For example: your each label file will look like this

traffic_sign 10.43654 34.2345 2.098 1.30955

but the yolo accepts only int as the class_id, not a string, so we have to replace the traffic_sign with an int in all the labels.

In [ ]:
import os

# Define the path to the directory containing the label files
label_dir = '/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k/labels/val'

# Function to replace "traffic sign" with 0 in a file
def replace_label(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    with open(file_path, 'w') as file:
        for line in lines:
            # Replace "traffic sign" with 0
            new_line = line.replace('traffic sign', '0')
            file.write(new_line)

# Iterate over all .txt files in the directory
for file_name in os.listdir(label_dir):
    if file_name.endswith('.txt'):
        file_path = os.path.join(label_dir, file_name)
        replace_label(file_path)

print("Label replacement completed.")


### Visualization
To view if the conversion from the BDD to yolo format is successful, we introduced a snippet that extracts the labels from the txt files and create bounding box on the images and then save them at desired directory. This way you can ensure that all the labels are correctly converted.

In [ ]:
import os
import cv2

def load_labels(label_file):
    with open(label_file, 'r') as file:
        labels = file.readlines()
    return [list(map(float, line.strip().split())) for line in labels]

def draw_bounding_boxes(image, labels, width, height):
    for label in labels:
        class_id, x_center, y_center, w, h = label
        x_center *= width
        y_center *= height
        w *= width
        h *= height
        x1 = int(x_center - w / 2)
        y1 = int(y_center - h / 2)
        x2 = int(x_center + w / 2)
        y2 = int(y_center + h / 2)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return image

def process_images(images_folder, labels_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for image_file in os.listdir(images_folder):
        if image_file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(images_folder, image_file)
            label_path = os.path.join(labels_folder, image_file.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt'))
            
            if os.path.exists(label_path):
                image = cv2.imread(image_path)
                height, width, _ = image.shape
                labels = load_labels(label_path)
                image_with_boxes = draw_bounding_boxes(image, labels, width, height)
                output_path = os.path.join(output_folder, image_file)
                cv2.imwrite(output_path, image_with_boxes)

images_folder = '/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k_cropped/images/100k/val/'
labels_folder = '/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k_cropped/labels/100k/val/'
output_folder = '/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k_view_boundingBox/val/'

process_images(images_folder, labels_folder, output_folder)

### Tiling images (OPTIONAL) 
If the object size are very small as compared to the image size(like traffic signs), then it is very difficult to detect them. For that a technique named Tiling is used, this method slice up the images into smaller chunks and adjusting the labels according with it. This way, the models accuracy drastically increase, but it only works well only if the object sizes are very small.

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image
from shapely.geometry import Polygon
from shutil import copyfile

def tiler(imnames, new_image_path, new_label_path, falsepath, slice_size, ext):
    for imname in imnames:
        im = Image.open(imname)
        imr = np.array(im, dtype=np.uint8)
        height = imr.shape[0]
        width = imr.shape[1]
        labname = imname.replace('images', 'labels').replace(ext, '.txt')
        labels = pd.read_csv(labname, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'])
        
        # Rescale coordinates from 0-1 to real image height and width
        labels[['x1', 'w']] = labels[['x1', 'w']] * width
        labels[['y1', 'h']] = labels[['y1', 'h']] * height
        
        boxes = []
        
        # Convert bounding boxes to shapely polygons
        for _, row in labels.iterrows():
            x1 = row['x1'] - row['w']/2
            y1 = (height - row['y1']) - row['h']/2
            x2 = row['x1'] + row['w']/2
            y2 = (height - row['y1']) + row['h']/2
            boxes.append((int(row['class']), Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])))
        
        counter = 0
        print('Image:', imname)
        
        # Create tiles and find intersection with bounding boxes for each tile
        for i in range((height // slice_size) + 1):
            for j in range((width // slice_size) + 1):
                x1 = j * slice_size
                y1 = height - (i * slice_size)
                x2 = ((j + 1) * slice_size) - 1
                y2 = (height - (i + 1) * slice_size) + 1
                pol = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
                imsaved = False
                slice_labels = []

                for box in boxes:
                    if pol.intersects(box[1]):
                        inter = pol.intersection(box[1])
                        
                        if not imsaved:
                            sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                            
                            # Check if the sliced image is not empty
                            if sliced.size == 0:
                                continue
                                
                            sliced_im = Image.fromarray(sliced)
                            filename = os.path.basename(imname)
                            slice_path = os.path.join(new_image_path, filename.replace(ext, f'_{i}_{j}{ext}'))                            
                            slice_labels_path = os.path.join(new_label_path, filename.replace(ext, f'_{i}_{j}.txt'))                            
                            print(slice_path)
                            sliced_im.save(slice_path)
                            imsaved = True                    
                        
                        # Get smallest rectangular polygon that contains the intersection
                        new_box = inter.envelope 
                        
                        # Get central point for the new bounding box 
                        centre = new_box.centroid
                        
                        # Get coordinates of polygon vertices
                        x, y = new_box.exterior.coords.xy
                        
                        # Get bounding box width and height normalized to slice size
                        new_width = (max(x) - min(x)) / slice_size
                        new_height = (max(y) - min(y)) / slice_size
                        
                        # Normalize central x and invert y for yolo format
                        new_x = (centre.coords.xy[0][0] - x1) / slice_size
                        new_y = (y1 - centre.coords.xy[1][0]) / slice_size
                        
                        counter += 1

                        slice_labels.append([box[0], new_x, new_y, new_width, new_height])
                
                if len(slice_labels) > 0:
                    slice_df = pd.DataFrame(slice_labels, columns=['class', 'x1', 'y1', 'w', 'h'])
                    print(slice_df)
                    slice_df.to_csv(slice_labels_path, sep=' ', index=False, header=False, float_format='%.6f')
                
                if not imsaved and falsepath:
                    sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                    
                    # Check if the sliced image is not empty
                    if sliced.size == 0:
                        continue
                    
                    sliced_im = Image.fromarray(sliced)
                    filename = os.path.basename(imname)
                    slice_path = os.path.join(falsepath, filename.replace(ext, f'_{i}_{j}{ext}'))                
                    sliced_im.save(slice_path)
                    print('Slice without boxes saved')
                    imsaved = True

def splitter(target, target_upfolder, ext):
    imnames = glob.glob(f'{target}/*{ext}')
    names = [os.path.basename(name) for name in imnames]

    # Split dataset for train and test
    train = []
    test = []
    for name in names:
        train.append(os.path.join(target, name))
    print('train:', len(train))
    print('test:', len(test))

    # Save train part
    with open(os.path.join(target_upfolder, 'train.txt'), 'w') as f:
        for item in train:
            f.write("%s\n" % item)

    # Save test part
    with open(os.path.join(target_upfolder, 'test.txt'), 'w') as f:
        for item in test:
            f.write("%s\n" % item)

# Define variables directly
source_images = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k_cropped/images/100k/train/"
source_labels = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k_cropped/labels/100k/train/"
target_images = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k_tiled/images/100k/train/"
target_labels = "/home/ubaurr/repositorio/traffic_signs/traffic_env/BDD_training/bdd100k_tiled/labels/100k/train/"
ext = ".jpg"
falsefolder = None
size = 160

# Create the target directories if they don't exist
os.makedirs(target_images, exist_ok=True)
os.makedirs(target_labels, exist_ok=True)

# Create classes.names file
classes_names_path = os.path.join(source_images, '../classes.names')
if not os.path.exists(classes_names_path):
    print('classes.names not found. Creating a default one.')
    with open(classes_names_path, 'w') as f:
        f.write('traffic sign\n')

# Copy classes.names file
upfolder = os.path.join(source_images, '..')
target_upfolder = os.path.join(target_images, '..')
copyfile(classes_names_path, os.path.join(target_upfolder, 'classes.names'))

if falsefolder:
    os.makedirs(falsefolder, exist_ok=True)

imnames = glob.glob(f'{source_images}/*{ext}')
labnames = glob.glob(f'{source_labels}/*.txt')

tiler(imnames, target_images, target_labels, falsefolder, size, ext)
splitter(target_images, target_upfolder, ext)
